In [ ]:
import pandas as pd
import json
import preprocessor as p
import tensorflow as tf
import gensim
import numpy as np
import re
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

In [ ]:
data = pd.read_csv('DigitalIndia-labelling-labelled.csv', encoding = 'utf-8')
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis =1)
data = data.reset_index(drop = True)
print(data.head())
print(data.shape)

In [ ]:
test_data = pd.read_csv('DI-labelling.csv', encoding = 'utf-8')
test_data = test_data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 8'], axis =1)
test_data = test_data.reset_index(drop = True)
print(test_data.head())
print(test_data.shape)

In [ ]:
unlabelled_data = pd.read_csv('FinalPreprocessed-DigitalIndia.csv', encoding = 'utf-8')
unlabelled_data = unlabelled_data.drop(['Unnamed: 0'], axis =1)
print(unlabelled_data.head())
print(unlabelled_data.shape)

In [ ]:
labels = []
i = 0
while i < data.shape[0]:
    pos = data['Positive'][i]
    neg = data['Negative'][i]
    temp_list = [pos , neg]
    labels.append(temp_list)
    i = i + 1

In [ ]:
test_labels = []
i = 0
while i < test_data.shape[0]:
    pos = test_data['Positive'][i]
    neg = test_data['Negative'][i]
    temp_list = [pos , neg]
    test_labels.append(temp_list)
    i = i + 1

In [ ]:
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
glove_model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [ ]:
max_sequence_length = 55
num_dimensions = 100
size = data.shape[0]
vector_output = np.zeros((size ,max_sequence_length, num_dimensions), dtype='float32')
i = 0    
while i < size:
    tweet_text = data['text'][i]
    cleanedLine = cleanSentences(tweet_text)
    split = cleanedLine.split()
    indexCounter = 0
    for word in split:
        try:
            vector_output[i][indexCounter] = glove_model[word]
        except KeyError:
            print(split[indexCounter])
        
        indexCounter = indexCounter + 1
    i = i + 1

In [ ]:
max_sequence_length = 55
num_dimensions = 100
size = unlabelled_data.shape[0]
vector_output_unlabelled = np.zeros((size ,max_sequence_length, num_dimensions), dtype='float32')
i = 0    
while i < size:
    tweet_text =unlabelled_data['text'][i]
    cleanedLine = cleanSentences(tweet_text)
    split = cleanedLine.split()
    indexCounter = 0
    for word in split:
        try:
            vector_output_unlabelled[i][indexCounter] = glove_model[word]
        except KeyError:
            print(split[indexCounter])
        
        indexCounter = indexCounter + 1
    i = i + 1

In [ ]:
max_sequence_length = 55
num_dimensions = 100
size = test_data.shape[0]
vector_output_test = np.zeros((size ,max_sequence_length, num_dimensions), dtype='float32')
i = 0    
while i < size:
    tweet_text = test_data['text'][i]
    cleanedLine = cleanSentences(tweet_text)
    split = cleanedLine.split()
    indexCounter = 0
    for word in split:
        try:
            vector_output_test[i][indexCounter] = glove_model[word]
        except KeyError:
            print(split[indexCounter])
        
        indexCounter = indexCounter + 1
    i = i + 1

In [ ]:
batch_size = 200
num_inputs = 55
num_dimensions = 100
num_outputs = 2
num_neurons = 30
training_iterations = 1000
learning_rate = 0.001

In [ ]:
tf.reset_default_graph()
#PLACEHOLDERS
X = tf.placeholder(dtype = tf.float32, shape = [None , num_inputs, num_dimensions], name = 'input_placeholder' )
y = tf.placeholder(dtype = tf.int32, shape = [None ,num_outputs], name = 'labels_placeholder')

In [ ]:
#RNN cell
cell = tf.contrib.rnn.BasicLSTMCell(num_units = num_neurons)
lstm_cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.85)
output, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [ ]:
output = tf.transpose(output, [1, 0, 2])
last = tf.gather(output, int(output.get_shape()[0]) - 1)
weight = tf.Variable(tf.truncated_normal([num_neurons, num_outputs]), name = 'weight')
bias = tf.Variable(tf.constant(0.1, shape=[num_outputs]), name ='bias')
prediction =tf.add(tf.matmul(last, weight), bias, name= 'prediction')
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
answer = tf.nn.softmax(prediction)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
train = optimizer.minimize(loss, name ='final_operation')
#Saving a model
saver = tf.train.Saver()
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("./outputModel", sess.graph)
    
    for iteration in range(500):
        sess.run(train, feed_dict = {X: vector_output, y: labels})
        
        if (iteration + 1) % 100 == 0:
            error = loss.eval(feed_dict = {X:vector_output , y: labels})
            print(iteration + 1, "\tError:", error)
                
    saver.save(sess, "./rnn_model", global_step=iteration)
    writer.close()
    
    #testing data
    print("Testset Accuracy")
    error = loss.eval(feed_dict = {X: vector_output_test , y: test_labels})
    print("\tError:", error)
    predictions_testset = answer.eval(feed_dict = {X: vector_output_test})
    
    #predicting unlabelled data
    unlabelled_answer = answer.eval(feed_dict={X:unlabelled_vector_output}, session=sess)
    print(unlabelled_answer.shape)
    

In [ ]:
pos_testlabel = []
neg_testlabel = []
i = 0
correct_counter = 0
wrong_counter = 0
while i < test_data.shape[0]:
    if predictions_testset[i][0] > predictions_testset[i][1]:
        pos_testlabel.append(1)
        neg_testlabel.append(0)
    else:
        pos_testlabel.append(0)
        neg_testlabel.append(1)
    i = i+ 1
#print(len(pos_testlabel))
#print(testBatchLabels)
i = 0
while i < test_data.shape[0]:
    if pos_testlabel[i] == test_labels[i][0]:
        correct_counter = correct_counter + 1
    else:
        wrong_counter = wrong_counter + 1
    i= i + 1
print(correct_counter)
print(wrong_counter)

In [ ]:
answer= pd.DataFrame()
answer['Positive'] = pos_testlabel
answer['Negative'] = neg_testlabel
answer.to_csv('answer.csv', index=False, encoding='utf-8')